# HST's Tranisting Exoplanet Noise Simulator 

This file demonstrates how to predict the:
    1. Transmission/emission spectrum S/N ratio
    2. Observation start window
for any system observed with WFC3/IR.

#### Background information
    Pandeia: ETC for JWST
    PandExo: Exoplanet noise simulator for JWST

In [1]:
import sys
sys.path.append('..')
import pandexo.engine.justdoit as jdi

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysynphot-0.9.14-py3.7-macosx-10.9-x86_64.egg/pysynphot/locations.py:346: UserWarning: Extinction files not found in /Users/arjunsavel/Desktop/research/atmospheres/grp/hst/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysynphot-0.9.14-py3.7-macosx-10.9-x86_64.egg/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /Users/arjunsavel/Desktop/research/atmospheres/grp/hst/cdbs/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysynphot-0.9.14-py3.7-macosx-10.9-x86_64.egg/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /Users/arjunsavel/Desktop/research/

## Edit Inputs

### Load in a blank exoplanet dictionary 

In [2]:
exo_dict = jdi.load_exo_dict()

#### Edit stellar and planet inputs

In [3]:
#WASP-43
exo_dict['star']['jmag']      = 9.995                # J magnitude of the system
exo_dict['star']['hmag']      = 9.397                # H magnitude of the system
#WASP-43b
exo_dict['planet']['type']    = 'user'               # user specified inputs
exo_dict['planet']['exopath'] = jdi.os.getcwd()+'/WASP43b-Eclipse_Spectrum.txt' # filename for model spectrum
exo_dict['planet']['w_unit']  = 'um'                 # wavelength unit
exo_dict['planet']['f_unit']  = 'fp/f*'              # flux ratio unit (can also put "rp^2/r*^2")
exo_dict['planet']['depth']   = 4.0e-3               # flux ratio
exo_dict['planet']['i']       = 82.6                 # Orbital inclination in degrees
exo_dict['planet']['ars']     = 5.13                 # Semi-major axis / stellar radius
exo_dict['planet']['period']  = 0.8135               # Orbital period in days   
exo_dict['planet']['transit_duration']= 4170.0/60/60/24#(optional if given above info) transit duration in days
exo_dict['planet']['w']       = 90                   #(optional) longitude of periastron. Default is 90
exo_dict['planet']['ecc']     = 0                    #(optional) eccentricity. Default is 0 

### Step 2) Load in instrument dictionary
 
- WFC3 G141
- WFC3 G102

In [4]:
inst_dict = jdi.load_mode_dict('WFC3 G141')

#### Edit HST/WFC3 detector and observation inputs

In [5]:
exo_dict['observation']['noccultations']               = 5            # Number of transits/eclipses
inst_dict['configuration']['detector']['subarray']     = 'GRISM256'   # GRISM256 or GRISM512
inst_dict['configuration']['detector']['nsamp']        = 10           # WFC3 N_SAMP, 1..15
inst_dict['configuration']['detector']['samp_seq']     = 'SPARS10'    # WFC3 SAMP_SEQ, SPARS5 or SPARS10
inst_dict['strategy']['norbits']                       = 3            # Number of HST orbits
inst_dict['strategy']['nchan']                         = 15           # Number of spectrophotometric channels
inst_dict['strategy']['scanDirection']                 = 'Forward'    # Spatial scan direction, Forward or Round Trip
inst_dict['strategy']['schedulability']                = 30           # 30 for small/medium program, 100 for large program
inst_dict['strategy']['windowSize']                    = 20           # (optional) Observation start window size in minutes. Default is 20 minutes.
inst_dict['strategy']['useFirstOrbit']                 = True         # (optional) Default is False, option to use first orbit
inst_dict['strategy']['calculateRamp']                 = True         # Enables ramp effect simulation for flux plot
inst_dict['strategy']['targetFluence']                 = 30000        # Maximum pixel fluence level (in electrons)

## Run PandExo

`    jdi.run_pandexo(exo, inst, param_space = 0, param_range = 0,save_file = True,
                            output_path=os.getcwd(), output_file = '')`

See wiki Attributes for more thorough explanation fo inputs


In [6]:
foo = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')
foo['wfc3_TExoNS']['info']

Running Single Case w/ User Instrument Dict
****WARNING: Number of specified HST orbits does not match number of recommended orbits: 4


{'Number of HST orbits': 3,
 'Use first orbit': True,
 'WFC3 parameters: NSAMP': 10,
 'WFC3 parameters: SAMP_SEQ': 'SPARS10',
 'Scan Direction': 'Forward',
 'Recommended scan rate (arcsec/s)': 0.06316627062891948,
 'Scan height (pixels)': 34.66131642820705,
 'Maximum pixel fluence (electrons)': 30000.0,
 'exposure time': 66.3965,
 'Estimated duty cycle (outside of Earth occultation)': 48.69076666666666,
 'Transit depth uncertainty(ppm)': 50.28678217034378,
 'Number of channels': 15,
 'Number of Transits': 5,
 'Start observations between orbital phases': '0.3890254729222154-0.40609847708802843'}

In [ ]:
inst_dict['configuration']['detector']['nsamp'] = None
inst_dict['configuration']['detector']['samp_seq'] = None
bar = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')
bar['wfc3_TExoNS']['info']

In [ ]:
inst_dict['strategy']['scanDirection'] = 'Round Trip'
hst = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')
hst['wfc3_TExoNS']['info']

## Plot Results


#### Plot simulated spectrum using specified file

In [ ]:
import pandexo.engine.justplotit as jpi 
#using foo from above
#other keys include model=True/False
datawave, dataspec, dataerror, modelwave, modelspec = jpi.hst_spec(foo)

## Compute earliest and latest possible start times for given start window size

In [ ]:
#using foo from above
obsphase1, obstr1, obsphase2, obstr2,rms = jpi.hst_time(foo)

# Compute simulated lightcurves in fluence (unit: electrons)
if configuration option `inst_dict['strategy']['calculateRamp']` is set as `True`, lightcurves will have ramp effect systematics

In [ ]:
obsphase1, counts1, obsphase2, counts2, noise = jpi.hst_simulated_lightcurve(foo)

## Print important info for observation

In [ ]:
hst['wfc3_TExoNS']['info']